# MIND Dataset Preprocessing

## Libraries, Imports, Setup

In [1]:
import pandas as pd
import numpy as np
import random
import string
import ast
import wordninja
import warnings
from scipy import sparse
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
pd.set_option('display.max_colwidth', None)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/christopherstephan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/christopherstephan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Loading Datasets

In [2]:
# Creating the path to the training dataset
behaviours_train_path = "/Users/christopherstephan/Documents/IE/Term 3/CAPSTONE/Datasets/MINDlarge_train/behaviors.tsv"  
news_train_path = "/Users/christopherstephan/Documents/IE/Term 3/CAPSTONE/Datasets/MINDlarge_train/news.tsv"

# Loading TSV file from the specified path
behaviours = pd.read_csv(behaviours_train_path, sep="\t", header=None)
news = pd.read_csv(news_train_path, sep="\t", header=None)

In [3]:
# Define the Column Names for the datasets
behaviours.columns = ["impression_id", "user_id", "time", "history", "impressions"]
news.columns = ["news_id", "category", "subcategory", "title", "abstract", "url", "title_entities", "abstract_entities"]

In [4]:
# Path to files
entity_embedding_path = "/Users/christopherstephan/Documents/IE/Term 3/CAPSTONE/Datasets/MINDlarge_train/entity_embedding.vec"  
relation_embedding_path = "/Users/christopherstephan/Documents/IE/Term 3/CAPSTONE/Datasets/MINDlarge_train/relation_embedding.vec"

with open(entity_embedding_path, "r") as f:
    for _ in range(5):  # Read first 5 lines
        print(f.readline())

with open(relation_embedding_path, "r") as f:
    for _ in range(5):  # Read first 5 lines
        print(f.readline())

Q34433	0.017808	-0.073256	0.102521	-0.059926	-0.060665	0.027027	-0.091728	-0.003057	-0.170798	0.111819	0.006821	-0.049873	-0.050532	-0.003127	-0.074472	-0.115891	-0.067093	-0.095272	0.019178	-0.083725	-0.060890	0.017744	0.049417	-0.026014	-0.048549	0.017528	0.044163	0.022111	-0.081519	0.046278	-0.183939	-0.063143	-0.014518	-0.080644	-0.099994	0.085905	-0.083003	-0.092844	-0.216481	0.125441	0.179819	0.036735	-0.085375	0.021276	-0.154971	0.039009	0.016059	0.067725	-0.148213	0.158773	-0.028527	0.125790	0.006361	0.067541	0.077552	0.060792	-0.044511	-0.005862	-0.068080	-0.063204	-0.094127	0.115441	-0.016472	0.106616	0.047839	-0.151805	-0.111083	-0.142330	-0.120680	-0.050393	-0.073787	0.017424	-0.081620	0.062599	-0.022102	-0.102688	-0.128149	-0.075895	0.095134	0.000984	0.010143	-0.068552	-0.026573	0.019735	-0.000981	-0.126635	0.008300	0.170557	0.002250	-0.157175	-0.077962	0.013433	0.045894	-0.071253	0.086445	-0.120466	0.059235	-0.071865	0.058854	0.024765	

Q41	-0.063388	-0.181451	0.057501	-0

In [5]:
# Load entity embeddings
entity_embeddings = {}
with open(entity_embedding_path, "r") as f:
    for line in f:
        values = line.strip().split()
        entity = values[0]  # First value is entity name
        vector = np.array(values[1:], dtype=np.float32)  # Rest are vector values
        entity_embeddings[entity] = vector


# Load relation embeddings
relation_embeddings = {}
with open(relation_embedding_path, "r") as f:
    for line in f:
        values = line.strip().split()
        entity = values[0]  # First value is entity name
        vector = np.array(values[1:], dtype=np.float32)  # Rest are vector values
        relation_embeddings[entity] = vector

## Cleaning, Preprocessing, Feature Engineering

### Behaviours

In [8]:
def clean_behaviours(behaviours):
    """
    Cleans the behaviors dataset by handling missing values and removing duplicates.

    Parameters:
        behaviours (pd.DataFrame): DataFrame containing the behaviors dataset.

    Returns:
        pd.DataFrame: Cleaned behaviors dataset.
    """
    behaviours["history"].fillna("No_History", inplace=True)

    # Remove duplicate rows
    behaviours.drop_duplicates(inplace=True)

    return behaviours

def convert_time(behaviours):
    """
    Converts the 'Time' column to datetime format and extracts the hour.

    Parameters:
        behaviours (pd.DataFrame): DataFrame containing the behaviors dataset.

    Returns:
        pd.DataFrame: Updated behaviors dataset with formatted datetime and extracted hour.
    """
    behaviours["time"] = pd.to_datetime(behaviours["time"], format="%m/%d/%Y %I:%M:%S %p", errors="coerce")
    behaviours["hour"] = behaviours["time"].dt.hour  # Extract hour
    return behaviours

def extract_num_clicks(behaviours):
    """
    Extracts the number of clicks per session based on the 'Impressions' column.

    Parameters:
        behaviours (pd.DataFrame): DataFrame containing the behaviors dataset.

    Returns:
        pd.DataFrame: Updated DataFrame with 'num_clicks' feature.
    """
    def count_clicks(impressions):
        """
        Counts the number of clicked news articles in an impression session.

        Parameters:
            impressions (str): String of impressions formatted as "N1-1 N2-0".

        Returns:
            int: Number of clicked items.
        """
        if not isinstance(impressions, str) or impressions.strip() == "":
            return 0
        return sum([int(i.split('-')[1]) for i in impressions.split() if '-' in i])

    behaviours["num_clicks"] = behaviours["impressions"].apply(count_clicks)

    return behaviours

def extract_history_length(behaviours):
    """
    Extracts the length of the user's reading history.

    Parameters:
        behaviours (pd.DataFrame): DataFrame containing the behaviors dataset.

    Returns:
        pd.DataFrame: Updated DataFrame with 'history_length' feature.
    """
    behaviours["history_length"] = behaviours["history"].apply(lambda x: 0 if x == "No_History" else len(x.split()))
    return behaviours

def split_clicked_nonclicked(behaviours):
    """
    Splits the impressions column into separate clicked and non-clicked news lists.

    Parameters:
        behaviours (pd.DataFrame): DataFrame containing the behaviors dataset.

    Returns:
        pd.DataFrame: Updated DataFrame with 'clicked_news' and 'non_clicked_news'.
    """
    def split_impressions(impressions):
        """
        Splits impressions into clicked and non-clicked news IDs.

        Parameters:
            impressions (str): String of impressions formatted as "N1-1 N2-0".

        Returns:
            tuple: Two lists (clicked_news, non_clicked_news).
        """
        clicked = []
        non_clicked = []
        if isinstance(impressions, str) and impressions.strip():
            for item in impressions.split():
                parts = item.split('-')
                if len(parts) == 2:
                    news_id, clicked_status = parts
                    if clicked_status == "1":
                        clicked.append(news_id)
                    else:
                        non_clicked.append(news_id)
        return clicked, non_clicked

    behaviours[["clicked_news", "non_clicked_news"]] = behaviours["impressions"].apply(lambda x: pd.Series(split_impressions(x)))
    return behaviours

def convert_user_id(behaviours):
    """
    Converts 'User ID' into a numerical index for easier model use.

    Parameters:
        behaviours (pd.DataFrame): DataFrame containing the behaviors dataset.

    Returns:
        pd.DataFrame: Updated DataFrame with numerical 'user_index'.
    """
    user_mapping = {uid: idx for idx, uid in enumerate(behaviours["user_id"].unique())}
    behaviours["user_index"] = behaviours["user_id"].map(user_mapping)
    return behaviours

def compute_user_avg_clicks(behaviours):
    """
    Computes the average number of clicks per user.

    Parameters:
        behaviours (pd.DataFrame): DataFrame containing the behaviors dataset.

    Returns:
        pd.DataFrame: Updated behaviors dataset with 'user_avg_clicks' feature.
    """
    user_click_counts = behaviours.groupby("user_id")["num_clicks"].sum()
    behaviours["user_avg_clicks"] = behaviours["user_id"].map(user_click_counts)
    return behaviours

def compute_recency(behaviours):
    """
    Computes recency of last interaction by measuring the time since the last recorded interaction.

    Parameters:
        behaviours (pd.DataFrame): DataFrame containing the behaviors dataset.

    Returns:
        pd.DataFrame: Updated behaviors dataset with 'recency' feature.
    """
    max_time = behaviours["time"].max()
    behaviours["recency"] = (max_time - behaviours["time"]).dt.total_seconds() / 3600  # Convert to hours
    return behaviours

In [9]:
def preprocess_behaviours(behaviours):
    """
    Full preprocessing pipeline for the behaviors dataset.

    Parameters:
        behaviours (pd.DataFrame): DataFrame containing the behaviors dataset.

    Returns:
        pd.DataFrame: Fully processed behaviors dataset.
    """
    behaviours = clean_behaviours(behaviours)
    behaviours = convert_time(behaviours)
    behaviours = extract_num_clicks(behaviours)
    behaviours = extract_history_length(behaviours)
    behaviours = split_clicked_nonclicked(behaviours)
    behaviours = convert_user_id(behaviours)
    behaviours = compute_user_avg_clicks(behaviours)
    behaviours = compute_recency(behaviours)

    return behaviours

In [10]:
# Run the full preprocessing pipeline
processed_behaviours = preprocess_behaviours(behaviours)

processed_behaviours.head()

,impression_id,user_id,time,history,impressions,hour,num_clicks,history_length,clicked_news,non_clicked_news,user_index,user_avg_clicks,recency
0,1,U87243,2019-11-10 11:30:54,N8668 N39081 N65259 N79529 N73408 N43615 N29379 N32031 N110232 N101921 N12614 N129591 N105760 N60457 N1229 N64932,N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N58258-0 N18478-0 N2591-0 N97778-0 N32954-0 N94157-1 N39404-0 N108809-0 N78699-1 N71090-1 N40282-0 N31174-1 N37924-0 N27822-0,11,4,16,"[N94157, N78699, N71090, N31174]","[N78206, N26368, N7578, N58592, N19858, N58258, N18478, N2591, N97778, N32954, N39404, N108809, N40282, N37924, N27822]",0,9,108.484722
1,2,U598644,2019-11-12 13:45:29,N56056 N8726 N70353 N67998 N83823 N111108 N107520 N53168 N78756 N106411 N81824 N85691 N16173 N24446 N120926 N19175 N126159 N26373 N72369 N73228 N111873 N88638 N29503 N31055,N47996-0 N82719-0 N117066-0 N8491-0 N123784-0 N21431-0 N106162-0 N59646-0 N118348-0 N25587-1 N37544-0 N20780-0 N36266-1 N46329-0 N64102-0 N120930-0 N62024-0 N11017-0 N76539-0 N26359-0 N108379-0 N87913-0 N125161-0 N23946-0 N83702-0 N62788-0 N6306-0 N66902-0 N93643-0,13,2,24,"[N25587, N36266]","[N47996, N82719, N117066, N8491, N123784, N21431, N106162, N59646, N118348, N37544, N20780, N46329, N64102, N120930, N62024, N11017, N76539, N26359, N108379, N87913, N125161, N23946, N83702, N62788, N6306, N66902, N93643]",1,11,58.241667
2,3,U532401,2019-11-13 11:23:03,N128643 N87446 N122948 N9375 N82348 N129412 N54948 N36094 N44660 N3948 N21332 N81364 N83062 N87788 N59280 N31323,N103852-0 N53474-0 N127836-0 N47925-1,11,1,16,[N47925],"[N103852, N53474, N127836]",2,4,36.615556
3,4,U593596,2019-11-12 12:24:09,N31043 N39592 N4104 N8223 N114581 N92747 N12070 N61321 N40052 N50176 N101119 N76810 N37509,N38902-0 N76434-0 N71593-0 N100073-0 N108736-0 N3061-0 N89658-0 N33030-0 N110473-0 N125655-0 N100206-0 N37544-0 N60370-0 N59935-0 N114935-1 N114135-0 N46262-0 N61065-0 N46329-0 N27368-0 N84381-0 N83412-0 N2624-0 N34269-0 N19053-0 N115724-0 N20990-0 N100380-0 N2074-0 N128367-0 N8719-0 N109344-0 N33078-0 N100539-0 N119665-0 N7937-0 N121580-0 N108581-0 N92498-0 N18947-0 N109642-0 N79048-0 N47277-0 N53225-0 N41710-0 N122819-0 N30417-0 N79044-0 N76539-0 N57763-0 N54239-0 N62203-0,12,1,13,[N114935],"[N38902, N76434, N71593, N100073, N108736, N3061, N89658, N33030, N110473, N125655, N100206, N37544, N60370, N59935, N114135, N46262, N61065, N46329, N27368, N84381, N83412, N2624, N34269, N19053, N115724, N20990, N100380, N2074, N128367, N8719, N109344, N33078, N100539, N119665, N7937, N121580, N108581, N92498, N18947, N109642, N79048, N47277, N53225, N41710, N122819, N30417, N79044, N76539, N57763, N54239, N62203]",3,10,59.597222
4,5,U239687,2019-11-14 20:03:01,N65250 N122359 N71723 N53796 N41663 N41484 N112765 N100765 N87642 N83576 N33881 N110863 N67875 N100466 N36761 N116312 N82374 N93136 N10659 N4857 N32369 N21104 N96120 N50191 N77001 N59349 N66535 N19719 N127567 N96764 N54850 N61319 N113547 N13277 N36604 N32800 N116279 N73179 N124109 N16608 N98215 N6229 N17492 N64552 N91231 N63676 N67779 N82799 N9375 N26586 N48453 N10711 N101263 N21773 N115066 N70097 N18031 N15471 N454 N56610 N112933 N75756 N72571 N41544 N48112 N42209 N44759 N76970 N69300 N108284 N76557 N104961 N46069 N81745 N117275 N12252 N116750 N18171 N87002 N60556 N45946 N20573 N6718 N50901 N3290 N10869 N40259 N14986 N97548 N51302 N39307 N86320 N89086 N29786 N37094 N38627 N51450 N107878 N126611 N57342 N81982 N87654 N3435 N83253 N123097 N113417 N71728 N100119 N57900 N7391 N13604 N71728 N111704 N3565 N53215 N39384 N44431 N129207 N31418 N32534 N124532 N15922 N118298 N19577 N34004 N104234 N103357 N7992 N87859 N80126 N86141 N4289 N127530 N36450 N91985 N9273 N105191 N2927 N49485 N34520 N129412 N75078 N97753 N88892 N79285 N115007 N68429 N56365 N60132 N91985 N79724 N32641 N29709 N36823 N96616 N29709 N46994 N128125 N100655 N18914 N43911 N119601 N43312 N4012 N102366 N92079 N71665 N54177 N43104 N21264 N44671 N98656 N108393 N94450 N75220 N101271 N59645 N55120 N128115 N262 N117511 N

### News

In [11]:
def clean_news(news):
    """
    Cleans the news dataset by handling missing values and duplicates.
    
    Parameters:
        news (pd.DataFrame): DataFrame containing the news dataset.
        
    Returns:
        pd.DataFrame: Cleaned news dataset.
    """
    # Remove rows with missing values (articles without an abstract)
    news.dropna(inplace=True)
    
    # Remove duplicate news articles
    news.drop_duplicates(inplace=True)

    return news

def preprocess_text(news):
    """
    Processes text fields in the news dataset by:
    - Splitting concatenated words in Category/SubCategory
    - Converting text to lowercase
    - Removing punctuation
    - Removing stopwords
    
    Parameters:
        news (pd.DataFrame): DataFrame containing the news dataset.
        
    Returns:
        pd.DataFrame: Updated news dataset with processed text.
    """
    def split_and_replace(word):
        return ' '.join(wordninja.split(word))

    # Apply function to Category and SubCategory
    news['category'] = news['category'].apply(split_and_replace)
    news['subcategory'] = news['subcategory'].apply(split_and_replace)

    # Convert text fields to lowercase
    columns_to_lower = ['category', 'subcategory', 'title', 'abstract']
    news[columns_to_lower] = news[columns_to_lower].applymap(lambda x: x.lower() if isinstance(x, str) else x)

    # Create 'Content' column by merging multiple text fields
    news['content'] = news[['category', 'subcategory', 'title', 'abstract']].apply(' '.join, axis=1)

    # Remove punctuation
    news['content'] = news['content'].str.replace(f"[{string.punctuation}]", "", regex=True)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    news['content'] = news['content'].apply(lambda text: " ".join(
        [word for word in text.split() if word.lower() not in stop_words]
    ))

    return news

def extract_word_count(news):
    """
    Adds a new feature 'Content_WC' that represents the word count in the content field.
    
    Parameters:
        news (pd.DataFrame): DataFrame containing the news dataset.
        
    Returns:
        pd.DataFrame: Updated news dataset with word count feature.
    """
    news['content_word_count'] = news['content'].str.split().str.len()
    return news

def extract_entities(news):
    """
    Extracts Wikidata entity IDs from the 'Title Entities' and 'Abstract Entities' columns.
    
    Parameters:
        news (pd.DataFrame): DataFrame containing the news dataset.
        
    Returns:
        pd.DataFrame: Updated news dataset with extracted entity IDs.
    """
    # Convert string representations of lists to actual lists
    news['title_entities'] = news['title_entities'].apply(ast.literal_eval)
    news['abstract_entities'] = news['abstract_entities'].apply(ast.literal_eval)

    # Extract Wikidata IDs from title and abstract entities
    news['title_wikidata_id'] = news['title_entities'].apply(lambda x: ' '.join([d['WikidataId'] for d in x]))
    news['abstract_wikidata_id'] = news['abstract_entities'].apply(lambda x: ' '.join([d['WikidataId'] for d in x]))

    # Combine both into a single column
    news['all_wikidata_ids'] = news['title_wikidata_id'] + ' ' + news['abstract_wikidata_id']
    
    return news

def extract_relations(news):
    """
    Extracts Wikidata relation (property) IDs from the 'Title Entities' and 'Abstract Entities' columns.

    Parameters:
        news (pd.DataFrame): DataFrame containing the news dataset.

    Returns:
        pd.DataFrame: Updated news dataset with extracted relation IDs.
    """
    news['title_relations'] = news['title_entities'].apply(lambda x: ' '.join([d['Type'] for d in x]))
    news['abstract_relations'] = news['abstract_entities'].apply(lambda x: ' '.join([d['Type'] for d in x]))

    # Combine into a single column
    news['all_relation_ids'] = news['title_relations'] + ' ' + news['abstract_relations']
    
    return news

In [12]:
def preprocess_news(news):
    """
    Full preprocessing pipeline for the news dataset.
    
    Parameters:
        news (pd.DataFrame): DataFrame containing the news dataset.
        embedding_file (str): Path to the entity embedding file.
        
    Returns:
        pd.DataFrame: Fully processed news dataset.
    """
    news = clean_news(news)
    news = preprocess_text(news)
    news = extract_word_count(news)
    news = extract_entities(news)
    news = extract_relations(news)

    return news

In [13]:
# Run the full preprocessing pipeline
processed_news = preprocess_news(news)

/var/folders/_8/hcj6x_gn51q577lb1lp9x6h40000gn/T/ipykernel_5528/1498567931.py:42: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  news[columns_to_lower] = news[columns_to_lower].applymap(lambda x: x.lower() if isinstance(x, str) else x)


### Entity Embeddings

In [15]:
def compute_entity_vectors(news, entity_embeddings):
    """
    Computes the average embedding vector for each news article based on extracted entity IDs.

    Parameters:
        news (pd.DataFrame): DataFrame containing the news dataset.
        entity_embeddings (dict): Dictionary mapping Wikidata IDs to embedding vectors.

    Returns:
        pd.DataFrame: Updated news dataset with 'average_vector' feature.
    """
    def calculate_average_vector(vector_ids, entity_embeddings):
        """
        Helper function to compute the mean vector for a given list of entity IDs.

        Parameters:
            vector_ids (str): Space-separated string of entity IDs.
            entity_embeddings (dict): Dictionary of pre-loaded entity embeddings.

        Returns:
            np.ndarray or np.nan: Averaged vector if entities exist, otherwise NaN.
        """
        vector_ids = vector_ids.split() if isinstance(vector_ids, str) else []
        vectors = [entity_embeddings.get(entity) for entity in vector_ids if entity in entity_embeddings]

        if vectors:
            return np.mean(vectors, axis=0)
        else:
            return np.nan  # Keep NaN for missing values

    # Apply function to each row in 'All Wikidata IDs' column
    news['entity_vector'] = news['all_wikidata_ids'].apply(lambda x: calculate_average_vector(x, entity_embeddings))

    return news


In [16]:
# Call the function
processed_news = compute_entity_vectors(processed_news, entity_embeddings)

### Relation Embeddings

In [17]:
def compute_relation_vectors(news, relation_embeddings):
    """
    Computes the average relation embedding vector for each news article.

    Parameters:
        news (pd.DataFrame): DataFrame containing the news dataset.
        relation_embeddings (dict): Dictionary mapping property IDs (Pxxx) to vectors.

    Returns:
        pd.DataFrame: Updated news dataset with 'relation_vector' feature.
    """
    def calculate_relation_vector(relation_ids, relation_embeddings):
        relation_ids = relation_ids.split() if isinstance(relation_ids, str) else []
        vectors = [relation_embeddings.get(r) for r in relation_ids if r in relation_embeddings]

        if vectors:
            return np.mean(vectors, axis=0)
        else:
            return np.nan  # Keep NaN for missing values

    # Apply function to each row in 'All Relation IDs' column
    news['relation_vector'] = news['all_relation_ids'].apply(lambda x: calculate_relation_vector(x, relation_embeddings))

    return news

In [18]:
# Call the function
processed_news = compute_relation_vectors(processed_news, relation_embeddings)

In [19]:
processed_news.head()

,news_id,category,subcategory,title,abstract,url,title_entities,abstract_entities,content,content_word_count,title_wikidata_id,abstract_wikidata_id,all_wikidata_ids,title_relations,abstract_relations,all_relation_ids,entity_vector,relation_vector
0,N88753,lifestyle,lifestyle royals,"the brands queen elizabeth, prince charles, and prince philip swear by","shop the notebooks, jackets, and more that the royals can't live without.",https://assets.msn.com/labs/mind/AAGH0ET.html,"[{'Label': 'Prince Philip, Duke of Edinburgh', 'Type': 'P', 'WikidataId': 'Q80976', 'Confidence': 1.0, 'OccurrenceOffsets': [48], 'SurfaceForms': ['Prince Philip']}, {'Label': 'Charles, Prince of Wales', 'Type': 'P', 'WikidataId': 'Q43274', 'Confidence': 1.0, 'OccurrenceOffsets': [28], 'SurfaceForms': ['Prince Charles']}, {'Label': 'Elizabeth II', 'Type': 'P', 'WikidataId': 'Q9682', 'Confidence': 0.97, 'OccurrenceOffsets': [11], 'SurfaceForms': ['Queen Elizabeth']}]",[],lifestyle lifestyle royals brands queen elizabeth prince charles prince philip swear shop notebooks jackets royals cant live without,18,Q80976 Q43274 Q9682,,Q80976 Q43274 Q9682,P P P,,P P P,"[0.0040573333, -0.03991733, -0.008374, 0.07914233, -0.020229666, -0.022212999, 0.040847663, 0.0014350001, 0.06182866, -0.031630997, 0.023040334, 0.04310767, 0.023592, -0.061693, 0.015099, -0.079108335, 0.028382666, 0.106527664, -0.008534334, -0.019020667, 0.011595999, -0.092747994, 0.07943733, -0.082109995, -0.070289664, -0.104519, 0.011144333, -0.009591, 0.044645667, -0.081025, -0.063390665, -0.020261666, 0.081290334, 0.031477336, -0.07450067, 0.012981, -0.0034046664, 0.010823333, -0.065875, 0.017982, -0.055929665, 0.08072767, -0.07246833, 0.025114998, -0.110455334, 0.041048, -0.011875999, -0.00434, -0.0027070008, -0.0019773333, 0.022897668, 0.066106, 0.010144333, 0.04570767, 0.044172335, 0.07795266, 0.014355333, 0.07451, -0.03834967, -0.011252667, 0.09210467, -0.077347666, 0.035256002, -0.058462333, 0.061819002, 0.08261534, 0.029722666, 0.029846, -0.035560668, 0.052448332, -0.0043619997, 0.026619999, -0.054742664, 0.00033166757, -0.019636666, -0.0045693335, 0.033881333, -0.021329336, 0.0013953336, 0.082415, 0.069087334, -0.0113659995, -0.00054833293, 0.013892, -0.011874001, -0.063751005, 0.07637099, 0.037085667, -0.0019800004, -0.027107665, -0.073342, 0.0146160005, 0.055785332, 0.024075666, 0.026349334, -0.052181665, 0.046933997, -0.024919666, 0.07169867, -0.036127668]",NaN
1,N45436,news,news science and technology,walmart slashes prices on last-generation ipads,apple's new ipad releases bring big deals on last year's models.,https://assets.msn.com/labs/mind/AABmf2I.html,"[{'Label': 'IPad', 'Type': 'J', 'WikidataId': 'Q2796', 'Confidence': 0.999, 'OccurrenceOffsets': [42], 'SurfaceForms': ['iPads']}, {'Label': 'Walmart', 'Type': 'O', 'WikidataId': 'Q483551', 'Confidence': 1.0, 'OccurrenceOffsets': [0], 'SurfaceForms': ['Walmart']}]","[{'Label': 'IPad', 'Type': 'J', 'WikidataId': 'Q2796', 'Confidence': 0.999, 'OccurrenceOffsets': [12], 'SurfaceForms': ['iPad']}, {'Label': 'Apple Inc.', 'Type': 'O', 'WikidataId': 'Q312', 'Confidence': 0.999, 'OccurrenceOffsets': [0], 'SurfaceForms': ['Apple']}]",news news science technology walmart slashes prices lastgeneration ipads apples new ipad releases bring big deals last years models,19,Q2796 Q483551,Q2796 Q312,Q2796 Q483551 Q2796 Q312,J O,J O,J O J O,"[0.011153751, 0.0022815005, 0.033238, -0.0060794977, -0.0118165, -0.0076985005, 0.02201475, -0.0164965, -0.01626675, 0.085608505, 0.0712205, 0.091341, 0.066110745, 0.016277503, -0.03425325, -0.020384248, -0.029653748, 0.05821325, -0.0339095, -0.048551247, -0.04525, -0.0260525, 0.0038565006, -0.025450751, 0.011129252, -0.01636625, -0.040598, -0.0368445, 0.029721, 0.027449999, -0.035932, 0.04201025, 0.03802625, 0.02801875, 0.03152425, -0.082879, 0.030481748, -0.068886, -0.04187625, 0.043128, -0.07453974, 0.07487525, 0.0071719997, 0.03655, -0.0570635, -0.09142325, 0.0098495, 0.037899252, -0.0417815, 0.083532505, -0.00

### Export Processed Data to Parquet

In [ ]:
# Save to Parquet format
processed_news.to_parquet("processed_news.parquet", index=False)
processed_behaviours.to_parquet("processed_behaviours.parquet", index=False)